In [7]:
import numpy as np
def checkSimilarity(node1, node2):
    #This is a function to calculate the similarity between two nodes
    #Importantly, all features MUST be normalized to values between 0 and 1
    params1 = np.array(node1.params)
    params2 = np.array(node2.params)
    diff = params1-params2 #Get the difference
    tot = np.sum(np.abs(diff))#Take the absolute value and sum to get total distance
    return tot
def calcValue(network,node):
    #Value of info is the total number of connections person has
    name = node.name
    connections = len(network.adjList[name])
    return connections
class Node:
    def __init__(self,name, params):
        #Node Structure
        self.name = name
        self.params = params
class Network:
    def __init__(self,n,a,MD):
        #Init a network with size n and inherent connection rate a
        self.adjList = {} #Adj list for representing graph
        self.nodeList = {} #list of added Nodes
        self.a = a
        self.n = n
        self.maxDist = MD #Number of params
        
    def addNode(self,name,params):
        #Add the node to the adjacency list
        self.adjList.update({name: []})
        #Add the node to the node list
        self.nodeList.update({name: Node(name,params)})
        for n,a in self.adjList.items():
            if(n != name): #Don't want node to connect to itself
                currNode = Node(name,params); #Get the node we are adding
                checkNode = self.nodeList[n] #Get the node we are seeing if it connects to
                tot = checkSimilarity(currNode,checkNode) #Calculate the similarity
                Q = 1-(tot+self.a*self.maxDist)/self.maxDist#Convert sim to a prob
                name1 = name #Get the name of the node being added
                name2 = checkNode.name #Get the name of the candidate connection
                self.addEdge(Q,name1,name2) #Check and see if the edge gets added

                
    def addEdge(self,Q,name1,name2):
        prob = (Q+self.a) #Add inherent prob to sim prob
        u = np.random.uniform(0,1) 
        if(u <= prob):
            #If the node is to be added, add to both parts of the adjacency list
            self.adjList[name2].append(name1)
            self.adjList[name1].append(name2)

n = 10
a = 0.2
network = Network(n,a,3)
for i in range(n):
    params = np.random.uniform(0,1,3)
    network.addNode(i,params)

IndentationError: expected an indented block (<ipython-input-7-f92895792059>, line 11)